<a href="https://colab.research.google.com/github/bdmarvin1/glitter-ads/blob/main/Google_Ads_Trend_Reporting_Multifamily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ==============================================================================
# SECTION 1: INSTALL REQUIRED LIBRARIES
# ==============================================================================
!pip install google-ads google-auth-oauthlib gspread python-dateutil google-api-python-client

# ==============================================================================
# SECTION 2: IMPORTS & AUTHENTICATION
# ==============================================================================
import gspread
from gspread.utils import rowcol_to_a1
from google.colab import auth, userdata
from google.auth import default
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException
import json
import time
import datetime
from dateutil.relativedelta import relativedelta
# --- NEW: Import for Google Docs API ---
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# --- UPDATED: Authenticate with all required scopes (Sheets, Drive, Docs) ---
auth.authenticate_user()
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents']
creds, _ = default(scopes=SCOPES)
gc = gspread.authorize(creds)


# --- Google Ads API Configuration ---
google_ads_client = None
try:
    print("--- Initializing Google Ads Client ---")
    credentials = {
        "developer_token": userdata.get('ADS_DEVELOPER_TOKEN'),
        "client_id": userdata.get('ADS_CLIENT_ID'),
        "client_secret": userdata.get('ADS_CLIENT_SECRET'),
        "refresh_token": userdata.get('ADS_REFRESH_TOKEN'),
        "login_customer_id": userdata.get('ADS_LOGIN_CUSTOMER_ID'),
        "use_proto_plus": True
    }
    google_ads_client = GoogleAdsClient.load_from_dict(credentials)
    print("✅ Google Ads client initialized successfully!")
except Exception as e:
    print(f"\n❌ An unexpected error occurred during setup: {e}")


# --- NEW: Google Docs API Configuration ---
DOCS_LOG_ID = '1ZMv52EXlCDNX0sWBDcQRNdvuOxRcdOogMy0jHGvVgTE'
docs_service = None
try:
    print("--- Initializing Google Docs Client ---")
    docs_service = build('docs', 'v1', credentials=creds)
    print("✅ Google Docs client initialized successfully!")
except HttpError as e:
    print(f"\n❌ An error occurred during Google Docs client setup: {e}")
except Exception as e:
    print(f"\n❌ An unexpected error occurred during setup: {e}")


# ==============================================================================
# SECTION 3: CONFIGURATION
# ==============================================================================
KEYWORD_GROUPS = {
    'Bedrooms': ['1 bed 1 bath apartments', '1 bed 1 bath for rent', '1 bed apartment for rent', '1 bed apartments near me', '1 bedroom and den apartments near me', '1 bedroom apartments', '1 bedroom apartments for rent', '1 bedroom apartments for rent near me', '1 bedroom apartments near me', '1 bedroom condo', '1 bedroom condo for rent', '1 bedroom condo for rent near me', '1 bedroom efficiency apartment for rent', '1 bedroom efficiency apartments near me', '1 bedroom for rent', '1 bedroom for rent near me', '1 bedroom studio for rent', '1 bedroom townhomes for rent', '1 bedroom townhouse', '1 br apartments near me', '1 room apartment for rent', '1bed 1bath for rent', '1bed 1bath for rent near me', '2 bdrm apt for rent', '2 bed 1 bath apartments', '2 bed 2 bath', '2 bed 2 bath apartments', '2 bed 2 bath apartments near me', '2 bed 2 bath for rent', '2 bed 2 bath townhomes for rent', '2 bed apartments', '2 bed apartments for rent', '2 bed apartments near me', '2 bed townhouse for rent', '2 bedroom 2 bath apartments', '2 bedroom apartments', '2 bedroom apartments for rent', '2 bedroom apartments for rent near me', '2 bedroom apartments near me', '2 bedroom condo for rent', '2 bedroom condo for rent near me', '2 bedroom for rent', '2 bedroom for rent near me', '2 bedroom property to rent', '2bedroom 2bath for rent', '3 bed apartments', '3 bed apartments near me', '3 bed townhouse for rent', '3 bedroom apartment complexes near me', '3 bedroom apartments', '3 bedroom apartments for rent', '3 bedroom apartments for rent near me', '3 bedroom apartments near me', '3 bedroom condo for rent', '3 bedroom condos for rent near me', '3 bedroom for rent', '3 bedroom for rent near me', '3 bedroom townhomes for rent', '3 bedroom townhomes for rent near me', '3 bedroom townhouses for rent', '3bedroom 2bath for rent', '4 bed 2 bath apartments', '4 bed townhouse for rent', '4 bedroom 2 bath apartments', '4 bedroom 4 bathroom apartment', '4 bedroom apartments', '4 bedroom apartments for rent', '4 bedroom apartments for rent near me', '4 bedroom apartments near me', '4 bedroom condo for rent', '4 bedroom townhomes for rent', '4 bedroom townhomes for rent near me', '4 bedroom townhouses for rent', '4 bedroom townhouses for rent near me', 'four bedroom apartments', 'modern 1 bedroom apartments', 'modern 2 bedroom apartment', 'modern one bedroom apartment', 'one bed apartment for rent', 'one bedroom apartment for rent', 'one bedroom apartment for rent near me', 'one bedroom apartments', 'one bedroom apartments near me', 'one bedroom condo', 'one bedroom condo for rent', 'one bedroom townhomes for rent', 'one room apartment for rent', 'three bedroom apartments for rent', 'three bedroom apartments near me', 'three bedroom condos for rent', 'three bedroom townhomes for rent', 'two bedroom apartments', 'two bedroom apartments near me', 'two bedroom condo for rent', 'two bedroom townhomes for rent', 'two bedroom townhouses for rent'],
    'Property Type': ['apartment townhomes for rent', 'apartment townhomes for rent near me', 'apartments', 'apartments and condos for rent', 'apartments and condos for rent near me', 'apartments and townhomes for rent', 'apartments and townhomes for rent near me', 'apartments and townhomes near me', 'bachelor apartment', 'bedroom studio', 'chicago industrial loft', 'chicago industrial lofts for rent', 'condo apartments for rent', 'condo townhomes for rent', 'condominium for rent', 'condominiums for rent near me', 'condos and townhomes for rent', 'condos and townhomes for rent near me', 'condos for lease', 'condos for lease near me', 'condos for rent', 'condos for rent by owner', 'condos for rent by owner near me', 'condos for rent near me', 'downtown apartments for rent', 'downtown condos for rent', 'duplex apartment for rent', 'duplex condo for rent', 'duplex for rent', 'duplex for rent near me', 'efficiencies for rent', 'efficiencies near me', 'efficiency apartment', 'efficiency apartment for rent', 'efficiency apartment near me', 'efficiency apartments near me', 'efficiency for rent', 'efficiency for rent near me', 'high rise apartments', 'high rise apartments for rent', 'high rise apartments near me', 'high rise condos for rent', 'houses and townhomes for rent', 'industrial apartments', 'industrial apartments for rent', 'industrial apartments near me', 'industrial loft apartments', 'industrial lofts', 'industrial lofts for rent', 'industrial studio apartment', 'industrial style apartments', 'industrial style lofts', 'industrial warehouse apartment', 'loft apartment', 'loft apartments for rent', 'loft apartments for rent near me', 'loft apartments near me', 'loft for rent', 'loft for rent near me', 'loft style apartments', 'loft style apartments near me', 'lofts near me', 'places to rent near me', 'private apartments for rent', 'private condos for rent', 'private owned apartments for rent', 'private owned townhomes for rent', 'private owned townhomes for rent near me', 'privately owned condos for rent', 'rentals near me', 'service apartment', 'service apartments near me', 'serviced apartments', 'single apartments near me', 'small apartments for rent', 'small studio apartment', 'studio apartment complexes near me', 'studio apartment for sale', 'studio apartments', 'studio apartments for rent', 'studio apartments for rent near me', 'studio apartments near me', 'studio apts for rent near me', 'studio apts near me', 'studio condo for rent', 'studio loft apartment', 'studios for rent', 'studios for rent near me', 'tiny apartment', 'town home for rent near me', 'town house rentals near me', 'townhome apartments', 'townhome apartments near me', 'townhome style apartments', 'townhomes and condos for rent by owner', 'townhomes and duplexes for rent', 'townhomes for lease near me', 'townhomes for rent', 'townhomes for rent by owner', 'townhomes for rent by private owner', 'townhomes for rent near me', 'townhomes near me', 'townhomes near me for rent by owner', 'townhouse apartments', 'townhouse apartments for rent', 'townhouse apartments near me', 'townhouse for lease near me', 'townhouses for rent', 'townhouses for rent near me', 'warehouse industrial loft apartment'],
    'Features': ['2 car garage with apartment', '2 car garage with loft apartment', '2 story garage with apartment', '24x24 garage with apartment cost', '4 car garage with apartment', 'above garage apartment for rent near me', 'all utilities included apartments near me', 'apartment complex with garage near me', 'apartment complexes near me with pools', 'apartment complexes with pools near me', 'apartments for rent by owner near me', 'apartments for rent near me pet friendly', 'apartments for rent pets allowed', 'apartments for rent that allow dogs', 'apartments for rent with utilities included', 'apartments near me with attached garages', 'apartments no breed restrictions', 'apartments that allow 3 pets near me', 'apartments that allow big dogs', 'apartments that allow dogs', 'apartments that allow dogs near me', 'apartments that allow large dogs', 'apartments that allow pets', 'apartments that allow pets near me', 'apartments that allow pit bulls', 'apartments that allow pit bulls near me', 'apartments with all utilities included', 'apartments with attached garages', 'apartments with attached garages for rent', 'apartments with attached garages near me', 'apartments with dog parks', 'apartments with garages', 'apartments with garages near me', 'apartments with utilities included', 'apartments with washer and dryer in unit near me', 'brand new apartment complexes near me', 'brand new apartments near me', 'cat friendly apartments near me', 'condos with garages for rent', 'condos with pools near me', 'detached garage with apartment', 'dog friendly apartment complexes near me', 'dog friendly apartments', 'dog friendly apartments for rent', 'dog friendly apartments for rent near me', 'dog friendly apartments near me', 'garage with apartment above', 'garage with loft apartment', 'gated apartment complex near me', 'gated apartments near me', 'large dog friendly apartments', 'metal garage with apartment', 'modern apartments near me', 'modern industrial apartments', 'modular garage with apartment', 'new apartment being built near me', 'new apartment buildings near me', 'new apartment complexes near me', 'new apartments for rent', 'new apartments for rent near me', 'new apartments for sale near me', 'new apartments near me', 'new apartments near me for rent', 'new construction apartments near me', 'new renovated apartments near me', 'new townhomes for rent', 'new townhomes for rent near me', 'newest apartments near me', 'newly built apartments near me', 'pet friendly apartment complexes near me', 'pet friendly apartments', 'pet friendly apartments for rent', 'pet friendly apartments near me', 'pet friendly apts near me', 'pet friendly condos for rent', 'pet friendly places for rent', 'pet friendly places for rent near me', 'pet friendly townhomes for rent', 'pet friendly townhomes for rent near me', 'places for rent that allow dogs', 'rentals near me pet friendly', 'student apartments near me', 'student housing near me', 'student living apartments near me', 'townhomes for rent near me pet friendly', 'townhomes with basement for rent', 'townhomes with garages for rent'],
    'Price': ['2nd chance apartments near me', 'affordable 2 bedroom apartments', 'affordable apartment complexes near me', 'affordable apartments', 'affordable apartments for rent', 'affordable apartments for rent near me', 'affordable apartments near me', 'affordable luxury apartments', 'affordable luxury apartments near me', 'affordable rental housing', 'affordable rentals near me', 'affordable studio apartments', 'affordable studio apartments near me', 'affordable townhomes for rent', 'affordable townhomes for rent near me', 'affordable townhomes near me', 'apartments for rent with move in specials', 'apartments with move in specials', 'apartments with move in specials near me', 'cheap 1 bedroom apartments', 'cheap 1 bedroom apartments for rent', 'cheap 1 bedroom apartments near me', 'cheap 2 bedroom apartments', 'cheap 2 bedroom apartments for rent near me', 'cheap 2 bedroom apartments near me', 'cheap 3 bedroom apartments', 'cheap 3 bedroom apartments near me', 'cheap 4 bedroom apartments', 'cheap affordable apartments', 'cheap apartment complexes near me', 'cheap apartments', 'cheap apartments for rent', 'cheap apartments for rent near me', 'cheap apartments for rent with utilities included near me', 'cheap apartments near me', 'cheap apts near me', 'cheap condos for rent', 'cheap condos for rent near me', 'cheap condos near me', 'cheap efficiency for rent near me', 'cheap lofts for rent near me', 'cheap luxury apartments', 'cheap luxury apartments near me', 'cheap one bedroom apartments', 'cheap pet friendly apartments', 'cheap pet friendly apartments near me', 'cheap rentals near me', 'cheap studio apartments', 'cheap studio apartments for rent', 'cheap studio apartments near me', 'cheap studios for rent', 'cheap studios for rent near me', 'cheap studios near me', 'cheap townhomes', 'cheap townhomes for rent', 'cheap townhomes for rent near me', 'cheap townhomes near me', 'efficiency for rent under $500', 'immediate move in specials near me', 'low income pet friendly apartments near me', 'low income studio apartments', 'low rent apartments near me', 'luxury apartment complex near me', 'luxury apartments', 'luxury apartments for rent', 'luxury apartments for rent near me', 'luxury apartments near me', 'luxury apts near me', 'luxury condos for rent', 'luxury condos near me', 'luxury high rise apartments', 'luxury loft apartments', 'luxury rentals near me', 'luxury studio apartments', 'luxury townhomes for rent', 'luxury townhomes for rent near me', 'luxury townhomes near me', 'modern luxury apartments', 'move in specials', 'move in specials near me', 'rent specials near me', 'rent to own condo', 'rent to own condos near me', 'rent to own townhomes near me', 'second chance apartments near me', 'second chance apartments with move in specials near me', 'second chance rentals near me', 'section 8 apartments', 'section 8 apartments near me', 'section 8 rentals'],
    'Lease Type': ['3 month lease apartments', '6 month lease apartments', 'apartments available now', 'apartments available now near me', 'apartments for lease near me', 'apartments for rent', 'apartments for rent month to month', 'apartments for rent near me', 'apartments month to month', 'apartments near me', 'apartments that lease to corporations', 'apt complex near me', 'apt for rent', 'available apartments near me', 'best apartment complexes near me', 'best apartments near me', 'corporate apartment rentals', 'corporate apartments', 'corporate apartments near me', 'corporate furnished apartments', 'corporate housing', 'corporate housing near me', 'corporate lease apartments', 'corporate lease apartments near me', 'corporate rentals', 'corporate short term rentals', 'extended stay apartments', 'extended stay apartments near me', 'extended stay rental', 'extended stay rentals near me', 'fully furnished apartment for rent', 'fully furnished apartments', 'fully furnished apartments for rent near me', 'fully furnished apartments near me', 'fully furnished condo for rent', 'fully furnished rentals', 'fully furnished rentals near me', 'furnished 1 bedroom apartment', 'furnished 1 bedroom apartment for rent', 'furnished apartments', 'furnished apartments for rent', 'furnished apartments for rent near me', 'furnished apartments for rent short term', 'furnished apartments near me', 'furnished apts for rent', 'furnished apts for rent near me', 'furnished condos for rent', 'furnished condos for rent near me', 'furnished corporate rentals near me', 'furnished efficiency apartments near me', 'furnished month to month rentals', 'furnished one bedroom apartments', 'furnished one bedroom apartments near me', 'furnished rentals near me', 'furnished short term rentals', 'furnished short term rentals near me', 'furnished studio apartments', 'furnished studio apartments for rent', 'furnished studio apartments near me', 'furnished studio for rent', 'furnished temporary housing', 'furnished townhomes for rent', 'good apartments near me', 'immediate move in apartments near me', 'local apartments for rent', 'long term corporate housing', 'month to month apartment lease', 'month to month apartment rentals', 'month to month apartment rentals near me', 'month to month apartments', 'month to month apartments near me', 'month to month furnished apartments', 'month to month furnished rentals', 'month to month lease apartments near me', 'month to month lease near me', 'month to month rent near me', 'month to month rentals', 'monthly furnished rentals', 'monthly rentals near me', 'nice apartments', 'nice apartments near me', 'nice townhomes for rent', 'rental month to month', 'short lease apartments near me', 'short stay apartments', 'short term apartment rentals', 'short term apartment rentals near me', 'short term apartments near me', 'short term condo rentals', 'short term corporate housing', 'short term fully furnished rentals', 'short term furnished apartments', 'short term house rentals', 'short term house rentals near me', 'short term housing', 'short term housing near me', 'short term lease', 'short term lease apartments', 'short term lease apartments near me', 'short term lease near me', 'short term monthly rentals', 'short term pet friendly rentals near me', 'short term property rental', 'short term rentals', 'short term rentals near me', 'temporary corporate housing', 'temporary furnished apartments']
}
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1pI2NEKXAjJhaqcUy-w4-4Wyas75B-6cg3MZMKVEBYkg/edit?gid=0#gid=0'
US_STATES = [
  "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

US_CITIES = [
  "New York, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", "Phoenix, AZ", "Philadelphia, PA", "San Antonio, TX", "San Diego, CA", "Dallas, TX", "Jacksonville, FL", "Fort Worth, TX", "San Jose, CA", "Austin, TX", "Charlotte, NC", "Columbus, OH", "Indianapolis, IN", "San Francisco, CA", "Seattle, WA", "Denver, CO", "Oklahoma City, OK", "Nashville, TN", "Washington, DC", "El Paso, TX", "Las Vegas, NV", "Boston, MA", "Detroit, MI", "Louisville, KY", "Portland, OR", "Memphis, TN", "Baltimore, MD", "Milwaukee, WI", "Albuquerque, NM", "Tucson, AZ", "Fresno, CA", "Sacramento, CA", "Atlanta, GA", "Mesa, AZ", "Kansas City, MO", "Raleigh, NC", "Colorado Springs, CO", "Omaha, NE", "Miami, FL", "Virginia Beach, VA", "Long Beach, CA", "Oakland, CA", "Minneapolis, MN", "Bakersfield, CA", "Tulsa, OK", "Tampa, FL", "Arlington, TX", "Aurora, CO", "Wichita, KS", "Cleveland, OH", "New Orleans, LA", "Henderson, NV", "Honolulu, HI", "Anaheim, CA", "Orlando, FL", "Lexington, KY", "Stockton, CA", "Riverside, CA", "Irvine, CA", "Corpus Christi, TX", "Newark, NJ", "Santa Ana, CA", "Cincinnati, OH", "Pittsburgh, PA", "Saint Paul, MN", "Greensboro, NC", "Jersey City, NJ", "Durham, NC", "Lincoln, NE", "North Las Vegas, NV", "Plano, TX", "Anchorage, AK", "Gilbert, AZ", "Madison, WI", "Reno, NV", "Chandler, AZ", "St. Louis, MO", "Chula Vista, CA", "Buffalo, NY", "Fort Wayne, IN", "Lubbock, TX", "St. Petersburg, FL", "Toledo, OH", "Laredo, TX", "Port St. Lucie, FL", "Glendale, AZ", "Irving, TX", "Winston-Salem, NC", "Chesapeake, VA", "Garland, TX", "Scottsdale, AZ", "Boise, ID", "Hialeah, FL", "Frisco, TX", "Richmond, VA", "Cape Coral, FL", "Norfolk, VA", "Spokane, WA", "Huntsville, AL", "Santa Clarita, CA", "Tacoma, WA", "Fremont, CA", "McKinney, TX", "San Bernardino, CA", "Baton Rouge, LA", "Modesto, CA", "Fontana, CA", "Salt Lake City, UT", "Moreno Valley, CA", "Des Moines, IA", "Worcester, MA", "Yonkers, NY", "Fayetteville, NC", "Sioux Falls, SD", "Grand Prairie, TX", "Rochester, NY", "Tallahassee, FL", "Little Rock, AR", "Amarillo, TX", "Overland Park, KS", "Columbus, GA", "Augusta, GA", "Mobile, AL", "Oxnard, CA", "Grand Rapids, MI", "Peoria, AZ", "Vancouver, WA", "Knoxville, TN", "Birmingham, AL", "Montgomery, AL", "Providence, RI", "Huntington Beach, CA", "Brownsville, TX", "Chattanooga, TN", "Fort Lauderdale, FL", "Tempe, AZ", "Akron, OH", "Glendale, CA", "Clarksville, TN", "Ontario, CA", "Newport News, VA", "Elk Grove, CA", "Cary, NC", "Aurora, IL", "Salem, OR", "Pembroke Pines, FL", "Eugene, OR", "Santa Rosa, CA", "Rancho Cucamonga, CA", "Shreveport, LA", "Garden Grove, CA", "Oceanside, CA", "Fort Collins, CO", "Springfield, MO", "Murfreesboro, TN", "Surprise, AZ", "Lancaster, CA", "Denton, TX", "Roseville, CA", "Palmdale, CA", "Corona, CA", "Salinas, CA", "Killeen, TX", "Paterson, NJ", "Alexandria, VA", "Hollywood, FL", "Hayward, CA", "Charleston, SC", "Macon, GA", "Lakewood, CO", "Sunnyvale, CA", "Kansas City, KS", "Springfield, MA", "Bellevue, WA", "Naperville, IL", "Joliet, IL", "Bridgeport, CT", "Mesquite, TX", "Pasadena, TX", "Olathe, KS", "Escondido, CA", "Savannah, GA", "McAllen, TX", "Gainesville, FL", "Pomona, CA", "Rockford, IL", "Thornton, CO", "Waco, TX", "Visalia, CA", "Syracuse, NY", "Columbia, SC", "Midland, TX", "Miramar, FL", "Palm Bay, FL", "Lakewood,NJ", "Jackson, MS", "Coral Springs, FL", "Victorville, CA", "Elizabeth, NJ", "Fullerton, CA", "Meridian, ID", "Torrance, CA", "Stamford, CT", "West Valley City, UT", "Orange, CA", "Cedar Rapids, IA", "Warren, MI", "Hampton, VA", "New Haven, CT", "Pasadena, CA", "Kent, WA", "Dayton, OH", "Fargo, ND", "Lewisville, TX", "Carrollton, TX", "Round Rock, TX", "Sterling Heights, MI", "Santa Clara, CA", "Norman, OK", "Columbia, MO", "Abilene, TX", "Pearland, TX", "Athens, GA", "College Station, TX", "Clovis, CA", "West Palm Beach, FL", "Allentown, PA", "North Charleston, SC", "Simi Valley, CA", "Topeka, KS", "Wilmington, NC", "Lakeland, FL", "Thousand Oaks, CA", "Concord, CA", "Rochester, MN", "Vallejo, CA", "Ann Arbor, MI", "Broken Arrow, OK", "Fairfield, CA", "Lafayette, LA", "Hartford, CT", "Arvada, CO", "Berkeley, CA", "Independence, MO", "Billings, MT", "Cambridge, MA", "Lowell, MA", "Odessa, TX", "High Point, NC", "League City, TX", "Antioch, CA", "Richardson, TX", "Goodyear, AZ", "Pompano Beach, FL", "Nampa, ID", "Menifee, CA", "Las Cruces, NM", "Clearwater, FL", "West Jordan, UT", "New Braunfels, TX", "Manchester, NH", "Miami Gardens, FL", "Waterbury, CT", "Provo, UT", "Evansville, IN", "Richmond, CA", "Westminster, CO", "Elgin, IL", "Conroe, TX", "Greeley, CO", "Lansing, MI", "Buckeye, AZ", "Tuscaloosa, AL", "Allen, TX", "Carlsbad, CA", "Everett, WA", "Springfield,IL", "Beaumont, TX", "Murrieta, CA", "Rio Rancho, NM", "Temecula, CA", "Concord, NC", "Tyler, TX", "Davie, FL", "South Fulton, GA", "Peoria, IL", "Sparks, NV", "Gresham, OR", "Santa Maria, CA", "Pueblo, CO", "Hillsboro, OR", "Edison, NJ", "Sugar Land, TX", "Ventura, CA", "Downey, CA", "Costa Mesa, CA", "Centennial, CO", "Edinburg, TX", "Spokane Valley, WA", "Jurupa Valley, CA", "Bend, OR", "West Covina, CA", "Boulder, CO", "Palm Coast, FL", "Lee's Summit, MO", "Dearborn, MI", "Green Bay, WI", "St. George, UT", "Woodbridge, NJ", "Brockton, MA", "Renton, WA", "Sandy Springs, GA", "Rialto, CA", "El Monte, CA", "Vacaville, CA", "Fishers, IN", "South Bend, IN", "Carmel, IN", "Yuma, AZ", "Burbank, CA", "Lynn, MA", "Quincy, MA", "El Cajon, CA", "Fayetteville, AR", "Suffolk, VA", "San Mateo, CA", "Chico, CA", "Inglewood, CA", "Wichita Falls, TX", "Boca Raton, FL", "Hesperia, CA", "Daly City, CA", "Clinton, MI", "Georgetown, TX", "New Bedford, MA", "Albany, NY", "Davenport, IA", "Plantation, FL", "Deltona, FL", "Federal Way, WA", "San Angelo, TX", "Tracy, CA", "Sunrise, FL"
]

CA_PROVINCES = ["Alberta", "British Columbia", "Ontario", "Quebec", "Manitoba", "New Brunswick", "Newfoundland and Labrador", "Nova Scotia", "Prince Edward Island", "Saskatchewan"]

CA_LOCATIONS = [ "Winnipeg, Manitoba", "Dieppe, New Brunswick", "Fredericton, New Brunswick", "Greater Lakeburn, New Brunswick", "Moncton, New Brunswick", "Saint John, New Brunswick", "Dartmouth, Nova Scotia", "Halifax, Nova Scotia", "Aurora, Ontario", "Barrie, Ontario", "Bowmanville, Ontario", "Chatham, Ontario", "Collingwood, Ontario", "East Gwillimbury, Ontario", "Goderich, Ontario", "Holland Landing, Ontario", "Leamington, Ontario", "London, Ontario", "Midland, Ontario", "Mississauga, Ontario", "Newmarket, Ontario", "Niagara Falls, Ontario", "North York, Ontario", "Oakville, Ontario", "Ottawa, Ontario", "Peterborough, Ontario", "Port Carling, Ontario", "Queensville, Ontario", "Richmond Hill, Ontario", "Seguin, Ontario", "St. Catharines, Ontario", "St. Marys, Ontario", "Sudbury, Ontario", "Thunder Bay, Ontario", "Toronto, Ontario", "Waterloo, Ontario", "Welland, Ontario", "Guelph, Ontario", "Scarborough, Ontario"]

LOCATIONS_TO_CHECK = (
    ["United States"]
    + US_STATES
    + US_CITIES
    + ["Canada"]
    + CA_PROVINCES
    + CA_LOCATIONS
)

# ==============================================================================
# SECTION 4: CORE LOGIC & HELPER FUNCTIONS
# ==============================================================================

def get_location_criterion_id(client, location_name):
    """Converts a location name into a Geo Target Constant Criterion ID."""
    gtc_service = client.get_service("GeoTargetConstantService")
    request = client.get_type("SuggestGeoTargetConstantsRequest")
    request.location_names.names.append(location_name)
    try:
        response = gtc_service.suggest_geo_target_constants(request)
        if not response.geo_target_constant_suggestions:
            print(f"    -> ⚠️  Could not find a location criterion for '{location_name}'.")
            return None
        return response.geo_target_constant_suggestions[0].geo_target_constant.resource_name
    except GoogleAdsException as ex:
        print(f"    -> ❌ Failed to get location ID for '{location_name}': {ex}")
        return None

def get_historical_metrics(client, customer_id, keywords, location_criterion):
    """
    Queries the Google Ads API for historical search volume data, starting from a fixed date (Jan 2023)
    up to the last full month.
    """
    print(f"  -> Fetching data for location: '{location_criterion}'...")

    request = client.get_type("GenerateKeywordHistoricalMetricsRequest")
    request.customer_id = customer_id
    request.keywords = keywords
    request.geo_target_constants.append(location_criterion)

    # --- UPDATED: Date Range Calculation ---
    # The end date is the last day of the previous month.
    today = datetime.date.today()
    last_full_month_date = today.replace(day=1) - relativedelta(days=1)
    end_year = last_full_month_date.year
    end_month_enum_value = getattr(client.get_type("MonthOfYearEnum").MonthOfYear, last_full_month_date.strftime('%B').upper())

    # The start date is now fixed to January 2023.
    start_year = 2023
    start_month_enum_value = client.get_type("MonthOfYearEnum").MonthOfYear.JANUARY

    print(f"    -> Requesting historical data from {start_year}-01 to {end_year}-{last_full_month_date.month:02d}.")

    request.historical_metrics_options.year_month_range.start.year = start_year
    request.historical_metrics_options.year_month_range.start.month = start_month_enum_value
    request.historical_metrics_options.year_month_range.end.year = end_year
    request.historical_metrics_options.year_month_range.end.month = end_month_enum_value

    try:
        time.sleep(1)
        keyword_plan_idea_service = client.get_service("KeywordPlanIdeaService")
        response = keyword_plan_idea_service.generate_keyword_historical_metrics(request=request)

        keyword_data = {}
        for result in response.results:
            monthly_volumes = {}
            for s in result.keyword_metrics.monthly_search_volumes:
                year = s.year
                month = s.month.value - 1

                if month == 0:
                    month = 12
                    year -= 1
                elif month > 12:
                    month -= 12
                    year += 1

                month_key = f"{year}-{month:02d}"
                monthly_volumes[month_key] = s.monthly_searches
            keyword_data[result.text] = monthly_volumes

        print(f"    -> Successfully fetched data for {len(keyword_data)} keywords.")
        return keyword_data
    except GoogleAdsException as ex:
        print(f'    -> ❌ Request failed: {ex.error.code().name}')
        for error in ex.failure.errors:
            print(f'       Error: {error.message} (Field: {error.location.field_path_elements[0].field_name})')
        return None
    except Exception as e:
        print(f"    -> ❌ An unexpected error occurred: {e}")
        return None

def aggregate_data(keyword_data, groups):
    """
    Aggregates search volume by the predefined groups (summing).
    This function now uses ALL unique months present in the API response data,
    with no upper limit on the number of months.
    """
    print("  -> Aggregating data...")

    # 1. Collect ALL unique YYYY-MM strings that actually exist in the API response.
    all_unique_months_from_api = set()
    for keyword_metrics in keyword_data.values():
        for month_key in keyword_metrics.keys():
            all_unique_months_from_api.add(month_key)

    # 2. Sort these months chronologically. This is crucial for correct sheet ordering.
    sorted_api_months = sorted(list(all_unique_months_from_api), key=lambda x: datetime.datetime.strptime(x, '%Y-%m'))

    # 3. Use all sorted months. The 25-month limit has been removed.
    final_months = sorted_api_months

    print(f"    -> Aggregating across {len(final_months)} total months.")

    # Initialize aggregated_volumes with the definitive month list (oldest to newest)
    aggregated_volumes = {group: {month: 0 for month in final_months} for group in groups}

    for group_name, keywords_in_group in groups.items():
        for keyword in keywords_in_group:
            if keyword in keyword_data:
                for month, volume in keyword_data[keyword].items():
                    # Check if the month is in our list (it always should be)
                    if month in final_months:
                        aggregated_volumes[group_name][month] += volume

    print("    -> Aggregation complete.")
    return aggregated_volumes, final_months

def update_google_sheet(spreadsheet_url, location_name, aggregated_data, months):
    """
    Updates the sheet by duplicating a template, clearing A1:X#, and pasting new data.
    Now dynamically adjusts for up to 25 months.
    """
    print(f"  -> Preparing to update sheet for '{location_name}'...")
    max_retries = 5
    for attempt in range(max_retries):
        try:
            spreadsheet = gc.open_by_url(spreadsheet_url)
            try:
                template_sheet = spreadsheet.worksheet("Sheet1")
            except gspread.exceptions.WorksheetNotFound:
                print("    -> ❌ CRITICAL ERROR: The template 'Sheet1' was not found."); return

            try:
                worksheet = spreadsheet.worksheet(location_name)
                print(f"    -> Found existing worksheet '{location_name}'.")
            except gspread.exceptions.WorksheetNotFound:
                print(f"    -> Worksheet '{location_name}' not found. Duplicating from 'Sheet1' at end of list...")

                source_sheet_id = template_sheet.id
                insert_index = len(spreadsheet.worksheets())

                requests = [{"duplicateSheet": {"sourceSheetId": source_sheet_id, "newSheetName": location_name, "insertSheetIndex": insert_index}}]
                spreadsheet.batch_update({"requests": requests})

                worksheet = spreadsheet.worksheet(location_name)
                print(f"    -> New worksheet '{location_name}' created and positioned at the end.")

            group_names = list(KEYWORD_GROUPS.keys())
            header_row = ['Keyword Group'] + months
            volume_rows = [[group] + [aggregated_data[group].get(month, 0) for month in months] for group in group_names]

            data_to_paste = [header_row] + volume_rows

            num_rows_to_clear = len(data_to_paste)
            num_cols_to_clear = len(data_to_paste[0])
            clear_end_cell = rowcol_to_a1(num_rows_to_clear, num_cols_to_clear)
            clear_range = f'A1:{clear_end_cell}'

            print(f"    -> Clearing target range: {clear_range}...")
            worksheet.batch_clear([clear_range])
            print("    -> Pasting monthly search volume data into A1.")
            worksheet.update('A1', data_to_paste, value_input_option='USER_ENTERED')

            worksheet.spreadsheet.batch_update({
                "requests": [{"autoResizeDimensions": {"dimensions": {"sheetId": worksheet.id, "dimension": "COLUMNS", "startIndex": 0, "endIndex": num_cols_to_clear}}}]
            })

            print(f"    -> ✅ Sheet '{location_name}' updated successfully!")
            return

        except gspread.exceptions.APIError as e:
            if e.response.status_code in [429, 500]:
                wait_time = 2 ** attempt
                print(f"    -> ⚠️ APIError (Status {e.response.status_code}). Retrying in {wait_time} seconds (attempt {attempt + 1}/{max_retries})...")
                time.sleep(wait_time)
            else:
                print(f"    -> ❌ An unrecoverable API error occurred while updating the Google Sheet: {e}")
                return
        except Exception as e:
            print(f"    -> ❌ An unexpected error occurred while updating the Google Sheet: {e}")
            return

    print(f"    -> ❌ Failed to update sheet '{location_name}' after {max_retries} attempts.")

def delete_all_except_specified_sheets(spreadsheet_url, sheets_to_keep):
    """
    Deletes all worksheets in a Google Sheet except those specified in sheets_to_keep.
    """
    print(f"\n--- Cleaning up Spreadsheet: Deleting unnecessary sheets ---")
    time.sleep(1)
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)
        all_worksheets = spreadsheet.worksheets()

        sheets_deleted = []
        for worksheet in all_worksheets:
            if worksheet.title not in sheets_to_keep:
                print(f"  -> Deleting sheet: '{worksheet.title}'...")
                time.sleep(.5)
                spreadsheet.del_worksheet(worksheet)
                sheets_deleted.append(worksheet.title)
            else:
                print(f"  -> Keeping sheet: '{worksheet.title}'")

        if sheets_deleted:
            print(f"✅ Successfully deleted: {', '.join(sheets_deleted)}")
        else:
            print("  -> No sheets to delete (all specified sheets were kept).")

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"❌ Error: Spreadsheet not found at URL: {spreadsheet_url}. Cannot clean up.")
    except Exception as e:
        print(f"❌ An error occurred during sheet cleanup: {e}")

# --- NEW: Google Docs Logging Functions ---
# --- CORRECTED: Google Docs Logging Functions ---
def get_doc_content_length(service, document_id):
    """
    Gets the insertion point at the end of the Google Doc.
    The API's endIndex is an exclusive upper bound, so we subtract 1
    to get the last valid insertion index.
    """
    try:
        doc = service.documents().get(documentId=document_id, fields='body(content(endIndex))').execute()
        # FIX: Subtract 1 from the endIndex to get the correct insertion point.
        return doc['body']['content'][-1]['endIndex'] - 1
    except HttpError as e:
        print(f"    -> ❌ Error getting document length: {e}")
        # Fallback for empty or erroring doc: 1 is the beginning of the document.
        return 1

def batch_update_doc(service, document_id, requests):
    """Applies a batch of update requests to the Google Doc."""
    if not requests:
        print("  -> No log data to send to Google Docs.")
        return
    try:
        service.documents().batchUpdate(
            documentId=document_id,
            body={'requests': requests}
        ).execute()
        print("  -> ✅ Successfully wrote logs to Google Doc.")
    except HttpError as e:
        print(f"    -> ❌ Error writing to Google Doc: {e}")


# ==============================================================================
# SECTION 5: MAIN EXECUTION (UPDATED WITH H2 LOG HEADER)
# ==============================================================================
def main():
    print("\n--- Starting Main Execution ---")
    if not google_ads_client or not docs_service:
        print("\n❌ Halting execution: A required client (Google Ads or Docs) is not initialized.")
        return

    all_keywords = [kw for group in KEYWORD_GROUPS.values() for kw in group]
    if not all_keywords:
        print("No keywords found. Please add keywords.")
        return

    # --- Initialize Logging with H2 Header ---
    log_requests = []
    run_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # We add a newline at the end for spacing after the header.
    header_text = f"{run_timestamp}\n"

    try:
        # Get the starting position for our new log entries.
        insert_at_index = get_doc_content_length(docs_service, DOCS_LOG_ID)-1
        if insert_at_index > 1: # Add a newline if we aren't at the very top
             header_text = "\n" + header_text
             insert_at_index += 1


        # 1. First request: Insert the header text.
        log_requests.append({
            'insertText': {
                'location': {'index': insert_at_index},
                'text': header_text
            }
        })

        # 2. Second request: Apply the 'Heading 2' style to the text we just inserted.
        log_requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': insert_at_index,
                    'endIndex': insert_at_index + len(header_text)
                },
                'paragraphStyle': {
                    'namedStyleType': 'HEADING_2'
                },
                # Specify the field we are updating.
                'fields': 'namedStyleType'
            }
        })

        # Send the batch containing both requests.
        batch_update_doc(docs_service, DOCS_LOG_ID, log_requests)

    except Exception as e:
        print(f"  -> ❌ Could not write initial header to Google Doc: {e}")


    # This list will hold the log strings for the current batch.
    log_batch = []

    # Optional: Enable sheet cleanup
    sheets_to_preserve = ['Sheet1', 'StateSummary']
#    delete_all_except_specified_sheets(SPREADSHEET_URL, sheets_to_preserve)

    customer_id = credentials['login_customer_id'].replace("-", "")

    for i, location_name_str in enumerate(LOCATIONS_TO_CHECK):

        # Determine the fully qualified location name for the API
        if location_name_str in US_CITIES or location_name_str in CA_LOCATIONS:
            # Already fully qualified, e.g., "Los Angeles, CA" or "Toronto, Ontario"
            qualified_location_name = location_name_str
        elif location_name_str in US_STATES:
            # e.g., "California" -> "California, United States"
            qualified_location_name = f"{location_name_str}, United States"
        elif location_name_str in CA_PROVINCES:
            # e.g., "Ontario" -> "Ontario, Canada"
            qualified_location_name = f"{location_name_str}, Canada"
        elif location_name_str == "United States" or location_name_str == "Canada":
            # Whole country
            qualified_location_name = location_name_str
        else:
            # Should not be reached, but as a fallback, use the name as is
            qualified_location_name = location_name_str

        print(f"\n--- Processing Location ({i+1}/{len(LOCATIONS_TO_CHECK)}): {location_name_str} (as '{qualified_location_name}') ---")

        log_entry = f"Processing: {location_name_str}... "

        location_criterion_resource_name = get_location_criterion_id(google_ads_client, qualified_location_name)

        if location_criterion_resource_name:
            keyword_metrics = get_historical_metrics(google_ads_client, customer_id, all_keywords, location_criterion_resource_name)
            if keyword_metrics:
                aggregated_data, months = aggregate_data(keyword_metrics, KEYWORD_GROUPS)
                update_google_sheet(SPREADSHEET_URL, location_name_str, aggregated_data, months)
                log_entry += "✅ Success\n"
            else:
                print(f"  -> Skipping sheet update for '{location_name_str}' due to data fetch failure.")
                log_entry += "❌ Failed (Data Fetch Error)\n"
        else:
            print(f"  -> Skipping '{location_name_str}' due to invalid location name.")
            log_entry += "❌ Failed (Invalid Location)\n"

        log_batch.append(log_entry)

        if len(log_batch) >= 10:
            print("\n--- Writing batch of 10 logs to Google Doc ---")
            full_log_text = "".join(log_batch)
            try:
                insert_at_index = get_doc_content_length(docs_service, DOCS_LOG_ID)
                requests = [{'insertText': {'location': {'index': insert_at_index}, 'text': full_log_text}}]
                batch_update_doc(docs_service, DOCS_LOG_ID, requests)
                log_batch.clear()
            except Exception as e:
                print(f"  -> ❌ Could not write log batch to Google Doc: {e}")

    if log_batch:
        print("\n--- Writing final batch of logs to Google Doc ---")
        full_log_text = "".join(log_batch)
        try:
            insert_at_index = get_doc_content_length(docs_service, DOCS_LOG_ID)
            requests = [{'insertText': {'location': {'index': insert_at_index}, 'text': full_log_text}}]
            batch_update_doc(docs_service, DOCS_LOG_ID, requests)
        except Exception as e:
            print(f"  -> ❌ Could not write final log batch to Google Doc: {e}")

    print("\n\n--- All operations complete. ---")
    print(f"View the results here: {SPREADSHEET_URL}")

--- Initializing Google Ads Client ---
✅ Google Ads client initialized successfully!
--- Initializing Google Docs Client ---
✅ Google Docs client initialized successfully!


In [ ]:
if __name__ == "__main__":
    main()


--- Starting Main Execution ---
  -> ✅ Successfully wrote logs to Google Doc.

--- Processing Location (1/447): United States (as 'United States') ---
  -> Fetching data for location: 'geoTargetConstants/2840'...
    -> Requesting historical data from 2023-01 to 2025-08.
    -> Successfully fetched data for 488 keywords.
  -> Aggregating data...
    -> Aggregating across 31 total months.
    -> Aggregation complete.
  -> Preparing to update sheet for 'United States'...
    -> Found existing worksheet 'United States'.
    -> Clearing target range: A1:AF6...
    -> Pasting monthly search volume data into A1.


/tmp/ipython-input-2292602410.py:257: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_paste, value_input_option='USER_ENTERED')


    -> ✅ Sheet 'United States' updated successfully!

--- Processing Location (2/447): Alabama (as 'Alabama, United States') ---
  -> Fetching data for location: 'geoTargetConstants/21133'...
    -> Requesting historical data from 2023-01 to 2025-08.
    -> Successfully fetched data for 488 keywords.
  -> Aggregating data...
    -> Aggregating across 31 total months.
    -> Aggregation complete.
  -> Preparing to update sheet for 'Alabama'...
    -> Found existing worksheet 'Alabama'.
    -> Clearing target range: A1:AF6...
    -> Pasting monthly search volume data into A1.
    -> ✅ Sheet 'Alabama' updated successfully!

--- Processing Location (3/447): Alaska (as 'Alaska, United States') ---
  -> Fetching data for location: 'geoTargetConstants/21132'...
    -> Requesting historical data from 2023-01 to 2025-08.
    -> Successfully fetched data for 488 keywords.
  -> Aggregating data...
    -> Aggregating across 31 total months.
    -> Aggregation complete.
  -> Preparing to update she

In [ ]:
# ==============================================================================
# SECTION 6: CUSTOM FUNCTIONS (State Summary - WITH CHANGE ANALYSIS)
# ==============================================================================

# IMPORTANT: Ensure SPREADSHEET_URL, KEYWORD_GROUPS, and US_STATES
# are defined in a previously executed cell before running this.

def create_state_summary_sheet(spreadsheet_url):
    "
    Creates/updates a 'StateSummary' sheet with states as rows and months as columns.
    It now intelligently appends summary columns for 1, 3, 6, 12, and 24-month
    change and percentage change, based on the available data.
    "
    print("\n--- Creating/Updating State Summary Sheet (with Change Analysis) ---")
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)

        try:
            summary_worksheet = spreadsheet.worksheet("StateSummary")
            print("  -> Found existing 'StateSummary' worksheet.")
        except gspread.exceptions.WorksheetNotFound:
            print("  -> 'StateSummary' worksheet not found. Creating it...")
            summary_worksheet = spreadsheet.add_worksheet(title="StateSummary", rows="100", cols="70", index=1)
            print("  -> New 'StateSummary' worksheet created.")

        months = []
        try:
            sample_sheet = spreadsheet.worksheet("United States")
            months = sample_sheet.row_values(1)[1:]
            if not months: raise ValueError("'United States' sheet is empty.")
            print(f"  -> Successfully loaded {len(months)} month headers from 'United States' sheet.")
        except (gspread.exceptions.WorksheetNotFound, ValueError, IndexError) as e:
            print(f"  -> WARNING: Could not get month headers ({e}). Generating fallback.")
            start_date = datetime.date(2023, 1, 1)
            end_date = datetime.date.today().replace(day=1) - relativedelta(days=1)
            current_date = start_date
            while current_date <= end_date:
                months.append(current_date.strftime('%Y-%m'))
                current_date += relativedelta(months=1)
            print(f"  -> Generated {len(months)} fallback month headers.")

        # --- Base data setup ---
        header_row = ['State'] + months
        data_rows = []
        all_sheet_titles = [s.title for s in spreadsheet.worksheets()]
        existing_state_sheets = [s for s in US_STATES if s in all_sheet_titles]

        if not existing_state_sheets:
            print("  -> No state-specific sheets found. Aborting summary creation.")
            return

        for state_name in existing_state_sheets:
            formula_row = [state_name]
            for j in range(len(months)):
                col_index = j + 2
                a1_notation = rowcol_to_a1(1, col_index)
                source_col_letter = ''.join(filter(str.isalpha, a1_notation))
                formula = f"='{state_name}'!{source_col_letter}7"
                formula_row.append(formula)
            data_rows.append(formula_row)

        # --- NEW: Add summary columns ---
        summary_periods = [('1m', 1), ('3m', 3), ('6m', 6), ('12m', 12), ('24m', 24)]
        num_months = len(months)
        last_month_col_index = 1 + num_months
        last_month_col_letter = ''.join(filter(str.isalpha, rowcol_to_a1(1, last_month_col_index)))

        percentage_format_requests = []

        for label, period_months in summary_periods:
            # Only add summary columns if we have enough historical data
            if num_months > period_months:
                print(f"  -> Adding {label} summary columns.")
                # Define new headers
                change_header = f"{label} Change"
                pct_change_header = f"{label} % Change"
                header_row.extend([change_header, pct_change_header])

                # Get the column for the base month (e.g., 12 months ago)
                base_month_col_index = last_month_col_index - period_months
                base_month_col_letter = ''.join(filter(str.isalpha, rowcol_to_a1(1, base_month_col_index)))

                # Get the column for the "Change" value we are about to add
                change_col_index = len(header_row) - 1
                change_col_letter = ''.join(filter(str.isalpha, rowcol_to_a1(1, change_col_index)))

                # Add the formulas to each data row
                for i, row in enumerate(data_rows):
                    sheet_row_num = i + 2 # +2 because sheet is 1-indexed and has a header

                    # Formula for absolute change: =(LastMonth - BaseMonth)
                    change_formula = f"={last_month_col_letter}{sheet_row_num}-{base_month_col_letter}{sheet_row_num}"

                    # Formula for % change: =IFERROR(Change / BaseMonth, 0)
                    pct_change_formula = f"=IFERROR({change_col_letter}{sheet_row_num}/{base_month_col_letter}{sheet_row_num}, 0)"

                    row.extend([change_formula, pct_change_formula])

                # Prepare a request to format the % change column
                pct_column_index = len(header_row) - 1 # 0-indexed column
                percentage_format_requests.append({
                    "repeatCell": {
                        "range": {"sheetId": summary_worksheet.id, "startColumnIndex": pct_column_index, "endColumnIndex": pct_column_index + 1},
                        "cell": {"userEnteredFormat": {"numberFormat": {"type": "PERCENT", "pattern": "0.00%"}}},
                        "fields": "userEnteredFormat.numberFormat"
                    }
                })

        # --- Combine all rows for final pasting ---
        final_rows_to_paste = [header_row] + data_rows

        # --- Update the sheet ---
        print(f"  -> Preparing to update sheet with {len(final_rows_to_paste)} rows and {len(header_row)} columns.")
        summary_worksheet.clear()
        summary_worksheet.update('A1', final_rows_to_paste, value_input_option='USER_ENTERED')

        # --- Apply all formatting requests and resize ---
        update_requests = [{
            "autoResizeDimensions": {"dimensions": {"sheetId": summary_worksheet.id, "dimension": "COLUMNS", "startIndex": 0, "endIndex": len(header_row)}}
        }] + percentage_format_requests

        summary_worksheet.spreadsheet.batch_update({"requests": update_requests})

        print("✅ 'StateSummary' sheet updated successfully with change analysis!")

    except Exception as e:
        print(f"❌ An error occurred while creating/updating the State Summary Sheet: {e}")


In [ ]:
# You would call this function in your main() after all individual location
# sheets have been processed. Example call in main():

create_state_summary_sheet(SPREADSHEET_URL)



--- Creating/Updating State Summary Sheet (with Change Analysis) ---
  -> Found existing 'StateSummary' worksheet.
  -> Successfully loaded 31 month headers from 'United States' sheet.
  -> Adding 1m summary columns.
  -> Adding 3m summary columns.
  -> Adding 6m summary columns.
  -> Adding 12m summary columns.
  -> Adding 24m summary columns.
  -> Preparing to update sheet with 51 rows and 42 columns.


/tmp/ipython-input-1105598732.py:115: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  summary_worksheet.update('A1', final_rows_to_paste, value_input_option='USER_ENTERED')


✅ 'StateSummary' sheet updated successfully with change analysis!


In [ ]:
# ==============================================================================
# SECTION 7: STANDALONE TEST FUNCTION (Save Raw API Response)
# ==============================================================================

# IMPORTANT: This function assumes your Google Ads API credentials
# (ADS_DEVELOPER_TOKEN, ADS_CLIENT_ID, etc.) are already set up in Colab's Secrets
# manager, as described in Section 2 of your main script.

# --- NEW/UPDATED IMPORTS REQUIRED ---
# No more json_format import needed for this simplified version
import json # Still needed if you use it elsewhere, but not for this specific dump
import datetime # Needed for date calculations
# ------------------------------------

def test_api_response_and_save(test_keyword: str, test_location_name: str, output_filename="raw_api_test_response.txt"): # Changed default extension to .txt
    "
    Makes a single Google Ads API call for a specific keyword and location,
    and saves the raw API response's string representation to a text file for debugging.

    Args:
        test_keyword (str): The single keyword to test.
        test_location_name (str): The location name (e.g., "United States", "Kansas").
        output_filename (str): The name of the text file to save the response to.
    "
    print(f"\n--- Running Standalone API Test for Keyword: '{test_keyword}' in '{test_location_name}' ---")

    google_ads_client_local = None
    try:
        credentials_local = {
            "developer_token": userdata.get('ADS_DEVELOPER_TOKEN'),
            "client_id": userdata.get('ADS_CLIENT_ID'),
            "client_secret": userdata.get('ADS_CLIENT_SECRET'),
            "refresh_token": userdata.get('ADS_REFRESH_TOKEN'),
            "login_customer_id": userdata.get('ADS_LOGIN_CUSTOMER_ID'),
            "use_proto_plus": True
        }
        google_ads_client_local = GoogleAdsClient.load_from_dict(credentials_local)
        print("  -> Google Ads client initialized for test.")
    except Exception as e:
        print(f"  ❌ Error initializing Google Ads client for test: {e}. Cannot proceed.")
        return

    customer_id_for_api = credentials_local['login_customer_id'].replace("-", "")

    # Get location criterion ID
    gtc_service = google_ads_client_local.get_service("GeoTargetConstantService")
    location_request = google_ads_client_local.get_type("SuggestGeoTargetConstantsRequest")
    location_request.location_names.names.append(test_location_name)

    location_criterion_resource_name = None
    try:
        location_response = gtc_service.suggest_geo_target_constants(location_request)
        if location_response.geo_target_constant_suggestions:
            location_criterion_resource_name = location_response.geo_target_constant_suggestions[0].geo_target_constant.resource_name
            print(f"  -> Found Criterion ID for '{test_location_name}': {location_criterion_resource_name}")
        else:
            print(f"  -> ⚠️ Could not find a location criterion for '{test_location_name}'. Cannot proceed.")
            return
    except GoogleAdsException as ex:
        print(f"  ❌ Failed to get location ID for '{test_location_name}': {ex.error.code().name}. Cannot proceed.")
        return
    except Exception as e:
        print(f"  ❌ An unexpected error occurred while getting location ID: {e}. Cannot proceed.")
        return

    # Define the historical metrics options to get the desired 25-month date range
    today = datetime.date.today()
    last_full_month_date = today.replace(day=1) - datetime.timedelta(days=1)
    end_year = last_full_month_date.year
    end_month_enum_value = getattr(google_ads_client_local.get_type("MonthOfYearEnum").MonthOfYear, last_full_month_date.strftime('%B').upper())

    start_dt = datetime.datetime(end_year, last_full_month_date.month, 1)
    for _ in range(24): # Go back 24 months from the end date for a total of 25 months
        year = start_dt.year
        month = start_dt.month - 1
        if month == 0:
            month = 12
            year -= 1
        start_dt = datetime.datetime(year, month, 1)

    start_year = start_dt.year
    start_month_enum_value = getattr(google_ads_client_local.get_type("MonthOfYearEnum").MonthOfYear, start_dt.strftime('%B').upper())

    print(f"  -> Requesting historical data from {start_year}-{start_dt.month:02d} to {end_year}-{last_full_month_date.month:02d} (total 25 months).")

    api_request = google_ads_client_local.get_type("GenerateKeywordHistoricalMetricsRequest")
    api_request.customer_id = customer_id_for_api
    api_request.keywords.append(test_keyword)
    api_request.geo_target_constants.append(location_criterion_resource_name)

    api_request.historical_metrics_options.year_month_range.start.year = start_year
    api_request.historical_metrics_options.year_month_range.start.month = start_month_enum_value
    api_request.historical_metrics_options.year_month_range.end.year = end_year
    api_request.historical_metrics_options.year_month_range.end.month = end_month_enum_value

    try:
        keyword_plan_idea_service = google_ads_client_local.get_service("KeywordPlanIdeaService")
        print(f"  -> Sending API request for keyword '{test_keyword}' in '{test_location_name}'...")
        raw_api_response = keyword_plan_idea_service.generate_keyword_historical_metrics(request=api_request)

        # --- FIX: Directly convert the raw response object to a string and save ---
        # This will save the __str__ representation (the one you saw printed to console).
        # It won't be proper JSON, but it will be the raw API response content.
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(str(raw_api_response)) # Convert to string and write directly

        print(f"  ✅ Raw API response (string representation) successfully saved to '{output_filename}'.")
        print(f"     You can view this file in Colab's file browser (left sidebar -> folder icon).")
        print("     NOTE: This content is the object's string representation, not formatted JSON.")
        print("\n--- Test complete. ---")

    except GoogleAdsException as ex:
        print(f"  ❌ API Request failed: {ex.error.code().name}.")
        for error in ex.failure.errors:
            print(f"    -> Error message: {error.message}")
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"      -> On field: {field_path_element.field_name}")
    except Exception as e:
        print(f"  ❌ An unexpected error occurred during API call: {e}")

# --- Example calls (uncomment to run in the cell) ---
# test_api_response_and_save("self storage unit", "United States")
# test_api_response_and_save("storage units", "Kansas")
# test_api_response_and_save("drive up storage", "Overland Park, Kansas", "drive_up_op_raw.txt")


In [ ]:
# ==============================================================================
# SECTION 8: TABLE OF CONTENTS GENERATION
# ==============================================================================
import gspread.utils
from gspread_formatting import *

def create_table_of_contents(spreadsheet_url):
    """
    Creates or updates a 'Table of Contents' sheet in the specified Google Sheet.

    The function gathers all existing sheets, categorizes them based on the
    pre-defined lists in the configuration, and generates a 4-column layout
    of hyperlinks to each sheet, with merged headers for each category.
    """
    print("\n--- Starting Table of Contents Generation ---")
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)
        print("  -> Successfully opened the spreadsheet.")
    except Exception as e:
        print(f"  -> ❌ Failed to open spreadsheet: {e}")
        return

    # 1. Gather all existing sheets and their properties (title and id)
    all_sheets = spreadsheet.worksheets()
    sheet_info = {sheet.title: sheet.id for sheet in all_sheets}
    print(f"  -> Found {len(all_sheets)} existing sheets.")

    # 2. Define the order and content of the ToC sections
    #    The key is the header that will be displayed.
    #    The value is the list of locations for that section.
    sections = {
        "United States": ["United States"],
        "U.S. States": US_STATES,
        "U.S. Cities": US_CITIES,
        "Canada": ["Canada"],
        "Canadian Provinces": CA_PROVINCES,
        "Canadian Locations": CA_LOCATIONS,
    }

    # 3. Categorize the existing sheets into the defined sections
    organized_sheets = {header: [] for header in sections.keys()}
    for sheet_name in sheet_info.keys():
        if sheet_name == "Table of Contents" or sheet_name == "Sheet1":
            continue # Skip the template and the ToC itself

        for header, location_list in sections.items():
            if sheet_name in location_list:
                organized_sheets[header].append(sheet_name)
                break

    # 4. Prepare data and formatting requests for the batch update
    data_to_write = []
    formatting_requests = []
    current_row_index = 1 # gspread is 1-indexed for ranges

    # Define cell formats
    header_format = CellFormat(
        backgroundColor=Color(0.9, 0.9, 0.9), # Light grey
        textFormat=TextFormat(bold=True, fontSize=12),
        horizontalAlignment='CENTER'
    )

    for header, sheet_names in organized_sheets.items():
        if not sheet_names:
            continue # Skip sections that have no corresponding sheets

        # --- Add Header Row ---
        data_to_write.append([header])
        header_range = f'A{current_row_index}:D{current_row_index}'
        formatting_requests.append(
            {'mergeCells': {
                'range': {'sheetId': None, 'startRowIndex': current_row_index - 1, 'endRowIndex': current_row_index, 'startColumnIndex': 0, 'endColumnIndex': 4},
                'mergeType': 'MERGE_ALL'
            }}
        )
        formatting_requests.append(
            {'repeatCell': {
                'range': {'sheetId': None, 'startRowIndex': current_row_index - 1, 'endRowIndex': current_row_index, 'startColumnIndex': 0, 'endColumnIndex': 4},
                'cell': {'userEnteredFormat': {'textFormat': {'bold': True}, 'horizontalAlignment': 'CENTER'}},
                'fields': 'userEnteredFormat(textFormat,horizontalAlignment)'
            }}
        )
        current_row_index += 1

        # --- Add Hyperlink Rows (in a 4-column grid) ---
        sheet_names.sort() # Sort alphabetically
        num_columns = 4
        for i in range(0, len(sheet_names), num_columns):
            row_data = []
            chunk = sheet_names[i:i + num_columns]
            for sheet_name in chunk:
                gid = sheet_info[sheet_name]
                # Formula to create a hyperlink to another sheet in the same document
                hyperlink_formula = f'=HYPERLINK("#gid={gid}","{sheet_name}")'
                row_data.append(hyperlink_formula)
            # Pad the row with empty strings if it's not full
            while len(row_data) < num_columns:
                row_data.append("")
            data_to_write.append(row_data)
        current_row_index += len(range(0, len(sheet_names), num_columns))


        # --- Add a blank row for spacing ---
        data_to_write.append(["", "", "", ""])
        current_row_index += 1


    # 5. Create or clear the 'Table of Contents' worksheet
    try:
        toc_sheet = spreadsheet.worksheet("Table of Contents")
        print("  -> Found existing 'Table of Contents' sheet. Clearing it.")
        toc_sheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        print("  -> 'Table of Contents' sheet not found. Creating a new one.")
        toc_sheet = spreadsheet.add_worksheet(title="Table of Contents", rows=len(data_to_write) + 50, cols=4)

    # Move ToC to the first position
    spreadsheet.reorder_worksheets([toc_sheet] + [s for s in all_sheets if s.id != toc_sheet.id])
    print("  -> Sheet positioned at the front.")

    # 6. Write the data and apply formatting
    print("  -> Writing data and applying formatting...")
    toc_sheet.update(
        'A1',
        data_to_write,
        value_input_option='USER_ENTERED' # IMPORTANT to correctly interpret formulas
    )

    # Add sheetId to all formatting requests
    for req in formatting_requests:
        if 'mergeCells' in req:
            req['mergeCells']['range']['sheetId'] = toc_sheet.id
        elif 'repeatCell' in req:
            req['repeatCell']['range']['sheetId'] = toc_sheet.id

    # Add column resize request
    formatting_requests.append(
        {"autoResizeDimensions": {
            "dimensions": {"sheetId": toc_sheet.id, "dimension": "COLUMNS", "startIndex": 0, "endIndex": 4}
        }}
    )

    if formatting_requests:
        spreadsheet.batch_update({'requests': formatting_requests})

    print("✅ --- Table of Contents generation complete! ---")


# --- Call the new function ---
create_table_of_contents(SPREADSHEET_URL)